In [1]:
import numba, llvmlite, umap

print("numba:", numba.__version__)
print("llvmlite:", llvmlite.__version__)
print("umap:", umap.__version__)


/home/tianqi/miniconda3/envs/fiftyone/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


numba: 0.59.1
llvmlite: 0.42.0
umap: 0.5.6


In [2]:
import fiftyone as fo

dataset_name = fo.list_datasets()[0]
if dataset_name is None:
    session = fo.launch_app(port=5151)
else:
    dataset = fo.load_dataset(dataset_name)
    print(f"Default dataset name: {dataset_name}")
    session = fo.launch_app(dataset,port=5151)

Default dataset name: 00_try


In [3]:
import fiftyone as fo
datasets = fo.list_datasets()
display(datasets)

['00_try',
 'jeff_0613-0624_04_ok',
 'lloyd_0715-0729_04_ok',
 'ms1_0605-0621_40_ok',
 'ms1_0710-0726_36_ok',
 'ms1_0726-0809_11_ok',
 'ms1_0809-0823_34_ok',
 'ms2_0726-0809_13_ok',
 'ms2_0809-0823_10_ok',
 'sw1_0605-0613_07_ok',
 'sw1_0711-0725_03_ok',
 'sw2_0808-0823_04_ok']

In [ ]:
# # fo.delete_sdataset("00_try") 
# # remove endwith patches
# for ds in datasets:
#     if ds.endswith("_patches"):
#         fo.delete_dataset(ds)   

In [ ]:
# for key in dataset.list_brain_runs():
#     dataset.delete_brain_run(key)

In [ ]:
for key in dataset.list_brain_runs():
    print(f"Deleting existing brain run: {key}")
    # dataset.delete_brain_run(key)

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

label_field = "01_swd_seg_results_coco"

# 模型只加载一次就行
model = foz.load_zoo_model("clip-vit-base32-torch")
# model = foz.load_zoo_model("dinov2-vits14-torch")
# model = foz.load_zoo_model("resnet50-imagenet-torch")
# model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")


for dataset_name in datasets:
    print(f"Dataset: {dataset_name}")
    dataset = fo.load_dataset(dataset_name)

    # 1) 对每个 ann 直接算 patch embedding（按 bbox/mask 裁剪，不导出图片）
    dataset.compute_patch_embeddings(
        model,
        patches_field=label_field,   # 关键：按这个字段里的 bbox/mask 作为 patch
        embeddings_field="emb_clip",      # embedding 存在每个 ann 的 .emb 里
    )

    # 2) 对所有 patch 做 PCA 可视化
    fob.compute_visualization(
        dataset,
        patches_field=label_field,   # 告诉 brain 这是 patch 字段
        embeddings="emb_clip",            # 用上一步算好的 embedding 字段
        method="umap",                # 先用 pca，规避 umap/numba 问题
        seed=51,
        brain_key="patches_clip_umap_v1",  # 每个 dataset 自己有一份同名 brain_key 就行
    )

    # 3) 对所有 patch 做 tsne 可视化
    fob.compute_visualization(
        dataset,
        patches_field=label_field,   # 告诉 brain 这是 patch 字段
        embeddings="emb_clip",            # 用上一步算好的 embedding 字段
        method="tsne",                # 先用 pca，规避 umap/numba 问题
        seed=51,
        brain_key="patches_clip_tsne_v1",  # 每个 dataset 自己有一份同名 brain_key 就行
    )

    # 4) 对所有 patch 做 umap 可视化
    fob.compute_visualization(
        dataset,
        patches_field=label_field,   # 告诉 brain 这是 patch 字段
        embeddings="emb_clip",            # 用上一步算好的 embedding 字段
        method="umap",                # 先用 pca，规避 umap/numba 问题
        seed=51,
        brain_key="patches_clip_umap_v1",  # 每个 dataset 自己有一份同名 brain_key 就行
    )


Dataset: 00_try


/home/tianqi/miniconda3/envs/fiftyone/lib/python3.10/site-packages/torch/cuda/__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5080 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_89 sm_90 compute_90.
If you want to use the NVIDIA GeForce RTX 5080 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


  50% |██████████-----------| 2/4 [1.6s elapsed, 1.6s remaining, 1.2 samples/s] 

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

label_field = "01_swd_seg_results_coco"

# 模型只加载一次就行
# model = foz.load_zoo_model("clip-vit-base32-torch")
model = foz.load_zoo_model("dinov2-vits14-torch")
# model = foz.load_zoo_model("resnet50-imagenet-torch")
# model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")


for dataset_name in datasets:
    print(f"Dataset: {dataset_name}")
    dataset = fo.load_dataset(dataset_name)

    # 1) 对每个 ann 直接算 patch embedding（按 bbox/mask 裁剪，不导出图片）
    dataset.compute_patch_embeddings(
        model,
        patches_field=label_field,   # 关键：按这个字段里的 bbox/mask 作为 patch
        embeddings_field="emb_dinov2",      # embedding 存在每个 ann 的 .emb 里
    )

    # 2) 对所有 patch 做 PCA 可视化
    fob.compute_visualization(
        dataset,
        patches_field=label_field,   # 告诉 brain 这是 patch 字段
        embeddings="emb_dinov2",            # 用上一步算好的 embedding 字段
        method="umap",                # 先用 pca，规避 umap/numba 问题
        seed=51,
        brain_key="patches_dinov2_umap_v1",  # 每个 dataset 自己有一份同名 brain_key 就行
    )

    # 3) 对所有 patch 做 tsne 可视化
    fob.compute_visualization(
        dataset,
        patches_field=label_field,   # 告诉 brain 这是 patch 字段
        embeddings="emb_dinov2",            # 用上一步算好的 embedding 字段
        method="tsne",                # 先用 pca，规避 umap/numba 问题
        seed=51,
        brain_key="patches_dinov2_tsne_v1",  # 每个 dataset 自己有一份同名 brain_key 就行
    )

    # 4) 对所有 patch 做 umap 可视化
    fob.compute_visualization(
        dataset,
        patches_field=label_field,   # 告诉 brain 这是 patch 字段
        embeddings="emb_dinov2",            # 用上一步算好的 embedding 字段
        method="umap",                # 先用 pca，规避 umap/numba 问题
        seed=51,
        brain_key="patches_dinov2_umap_v1",  # 每个 dataset 自己有一份同名 brain_key 就行
    )


In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

label_field = "01_swd_seg_results_coco"

# 模型只加载一次就行
# model = foz.load_zoo_model("clip-vit-base32-torch")
# model = foz.load_zoo_model("dinov2-vits14-torch")
model = foz.load_zoo_model("resnet50-imagenet-torch")
# model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")


for dataset_name in datasets:
    print(f"Dataset: {dataset_name}")
    dataset = fo.load_dataset(dataset_name)

    # 1) 对每个 ann 直接算 patch embedding（按 bbox/mask 裁剪，不导出图片）
    dataset.compute_patch_embeddings(
        model,
        patches_field=label_field,   # 关键：按这个字段里的 bbox/mask 作为 patch
        embeddings_field="emb_resnet50",      # embedding 存在每个 ann 的 .emb 里
    )

    # 2) 对所有 patch 做 PCA 可视化
    fob.compute_visualization(
        dataset,
        patches_field=label_field,   # 告诉 brain 这是 patch 字段
        embeddings="emb_resnet50",            # 用上一步算好的 embedding 字段
        method="umap",                # 先用 pca，规避 umap/numba 问题
        seed=51,
        brain_key="patches_resnet50_umap_v1",  # 每个 dataset 自己有一份同名 brain_key 就行
    )

    # 3) 对所有 patch 做 tsne 可视化
    fob.compute_visualization(
        dataset,
        patches_field=label_field,   # 告诉 brain 这是 patch 字段
        embeddings="emb_resnet50",            # 用上一步算好的 embedding 字段
        method="tsne",                # 先用 pca，规避 umap/numba 问题
        seed=51,
        brain_key="patches_resnet50_tsne_v1",  # 每个 dataset 自己有一份同名 brain_key 就行
    )

    # 4) 对所有 patch 做 umap 可视化
    fob.compute_visualization(
        dataset,
        patches_field=label_field,   # 告诉 brain 这是 patch 字段
        embeddings="emb_resnet50",            # 用上一步算好的 embedding 字段
        method="umap",                # 先用 pca，规避 umap/numba 问题
        seed=51,
        brain_key="patches_resnet50_umap_v1",  # 每个 dataset 自己有一份同名 brain_key 就行
    )


In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

label_field = "01_swd_seg_results_coco"

# 模型只加载一次就行
# model = foz.load_zoo_model("clip-vit-base32-torch")
# model = foz.load_zoo_model("dinov2-vits14-torch")
# model = foz.load_zoo_model("resnet50-imagenet-torch")
model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")


for dataset_name in datasets:
    print(f"Dataset: {dataset_name}")
    dataset = fo.load_dataset(dataset_name)

    # 1) 对每个 ann 直接算 patch embedding（按 bbox/mask 裁剪，不导出图片）
    dataset.compute_patch_embeddings(
        model,
        patches_field=label_field,   # 关键：按这个字段里的 bbox/mask 作为 patch
        embeddings_field="emb_mobilenet",      # embedding 存在每个 ann 的 .emb 里
    )

    # 2) 对所有 patch 做 PCA 可视化
    fob.compute_visualization(
        dataset,
        patches_field=label_field,   # 告诉 brain 这是 patch 字段
        embeddings="emb_mobilenet",            # 用上一步算好的 embedding 字段
        method="umap",                # 先用 pca，规避 umap/numba 问题
        seed=51,
        brain_key="patches_mobilenet_umap_v1",  # 每个 dataset 自己有一份同名 brain_key 就行
    )

    # 3) 对所有 patch 做 tsne 可视化
    fob.compute_visualization(
        dataset,
        patches_field=label_field,   # 告诉 brain 这是 patch 字段
        embeddings="emb_mobilenet",            # 用上一步算好的 embedding 字段
        method="tsne",                # 先用 pca，规避 umap/numba 问题
        seed=51,
        brain_key="patches_mobilenet_tsne_v1",  # 每个 dataset 自己有一份同名 brain_key 就行
    )

    # 4) 对所有 patch 做 umap 可视化
    fob.compute_visualization(
        dataset,
        patches_field=label_field,   # 告诉 brain 这是 patch 字段
        embeddings="emb_mobilenet",            # 用上一步算好的 embedding 字段
        method="umap",                # 先用 pca，规避 umap/numba 问题
        seed=51,
        brain_key="patches_mobilenet_umap_v1",  # 每个 dataset 自己有一份同名 brain_key 就行
    )


# 导出

In [ ]:
# from fiftyone import ViewField as F

# label_field = "01_swd_seg_results_coco"   # 你的检测/分割字段
# export_dir = "/home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/01_data/00_test/00_try/del"  # 换成自己的路径

# view = session.view   # 💡 当前 App 的视图（包含你所有筛选）

# view.export(
#     export_dir=export_dir,
#     dataset_type=fot.COCODetectionDataset,  # 或 COCOInstancesDataset
#     label_field=label_field,
#     export_media=True,                      # 要不要把图也导出去
# )
